In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "../data/25년_수확량 통계.csv"
# xls = pd.ExcelFile(file_path)
# sheet_names = xls.sheet_names
# print(sheet_names)

df= pd.read_csv(file_path)


print("--- [시작] 데이터 전처리(공백 및 타입 정리)를 시작합니다. ---")

# 2. [문자형] 컬럼 공백 제거 (앞/뒤 공백)
# 분석에 주로 사용되는 범주형 컬럼들
categorical_cols = [
    '지역', '코드번호', '경작자', '구분', 'CASE', '시비 처리', '작물', '품종'
]

for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# 2-1. [문자형] '품종' 컬럼의 *중간* 공백도 모두 제거
if '품종' in df.columns:
    df['품종'] = df['품종'].str.replace(r'\s+', '', regex=True)

print("범주형(문자) 컬럼 공백 정리 완료.")

# 3. [숫자형] 문자열로 저장된 숫자 컬럼 정리
# (쉼표, 공백이 포함된 컬럼들)
numeric_object_cols = [
    '면적(m2)', '면적(평)', '수확량(kg)', '건중량(kg)',
    '평당 수확량', '건조 수확량(kg/10a)', '총 질소 살포량(kg/10a)'
]

for col in numeric_object_cols:
    if col in df.columns:
        # 1. 문자로 변환 (결측치는 'nan' 문자열이 됨)
        s = df[col].astype(str)
        # 2. 쉼표(,) 제거
        s = s.str.replace(',', '', regex=False)
        # 3. 모든 공백(앞/뒤/중간) 제거
        s = s.str.replace(r'\s+', '', regex=True)

        # 4. 숫자로 변환 (변환 안되는 값은 NaN 처리)
        df[col] = pd.to_numeric(s, errors='coerce')

print("숫자형(문자) 컬럼 숫자 타입 변환 완료.")

print("--- [완료] 데이터 전처리 완료. ---")

# 4. [확인] 전처리 후 데이터 정보 출력
print("\n--- [처리]")

print(df.head())

--- [시작] 데이터 전처리(공백 및 타입 정리)를 시작합니다. ---
범주형(문자) 컬럼 공백 정리 완료.
숫자형(문자) 컬럼 숫자 타입 변환 완료.
--- [완료] 데이터 전처리 완료. ---

--- [처리]
   지역    년도   코드번호                         주소  경작자  면적(m2)   면적(평)  구분 CASE  \
0  김제  2025  GJ-R1  전북특별자치도 김제시 부량면 용성리 22-11  장수용  4000.0  1210.0  실증    1   
1  김제  2025  GJ-R2     전북특별자치도 김제시 부량면 신용리 12  장수용  4000.0  1210.0  실증    1   
2  김제  2025  GJ-R3   전북특별자치도 김제시 부량면 신용리 12-1  장수용  4000.0  1210.0  실증    2   
3  김제  2025  GJ-R4   전북특별자치도 김제시 부량면 신용리 12-2  장수용  3980.0  1204.0  실증    2   
4  김제  2025  GJ-R5   전북특별자치도 김제시 부량면 신용리 12-3  장수용  4000.0  1210.0  실증    2   

            시비 처리  ... 밑거름 살포수량(포대) 밑거름 포대당 무게(kg)  포대당 질소 무게(kg)  \
0  밑거름 정량, 웃거름 변량  ...         11.0           15.0           4.95   
1  밑거름 정량, 웃거름 변량  ...         11.0           15.0           4.95   
2  밑거름 정량, 웃거름 변량  ...         11.0           15.0           4.95   
3  밑거름 정량, 웃거름 변량  ...         10.0           15.0           4.95   
4  밑거름 정량, 웃거름 변량  ...         11.0           15.0         

In [3]:
import plotly.express as px
import pandas as pd

# ---------------------------------------------------------
# 1. 데이터 로드 및 진천 지역 필터링
# ---------------------------------------------------------
# df = pd.read_csv("25년_수확량 통계.csv") # 데이터가 이미 로드되어 있다고 가정
# df_jincheon = df[df['지역'] == '진천'].copy()  # 원본 보존을 위해 copy() 사용

# (테스트용 데이터 생성 - 실제 사용 시 위 주석을 풀고 df를 사용하세요)
# 만약 df가 정의되지 않았다면 이 부분은 무시하고 위 코드를 사용하세요.
if 'df' not in locals():
    df = pd.read_csv("25년_수확량 통계.csv")

# 진천 지역만 필터링
df_jc = df[df['지역'] == '진천'].copy()

# ---------------------------------------------------------
# 2. 데이터 전처리 (결측치 처리 및 구분 컬럼 생성)
# ---------------------------------------------------------

# (1) 진천 농협의 필지코드(N/a) 처리
# 경작자가 '농협 평균'이거나 코드가 비어있는 경우 '진천 농협'으로 채움
mask_nonghyup = (df_jc['경작자'] == '농협 평균') | (df_jc['코드번호'].isna()) | (df_jc['코드번호'].astype(str) == 'nan')
df_jc.loc[mask_nonghyup, '코드번호'] = '진천 농협'

# (2) 색상 구분을 위한 '카테고리' 컬럼 생성
# 로직: 농협은 '비교군'으로, 나머지는 '품종'으로 설정
def get_category(row):
    if row['코드번호'] == '진천 농협':
        return '진천 농협 (비교군)'
    else:
        return row['품종']

df_jc['구분'] = df_jc.apply(get_category, axis=1)

# (3) X축 레이블 생성 (필지코드 + 경작자 병기)
# 경작자 구분이 필요하므로, 그래프 X축에 경작자 이름을 같이 표시
def get_xlabel(row):
    if row['코드번호'] == '진천 농협':
        return '진천 농협<br>(평균)' # 줄바꿈(<br>) 적용
    else:
        cultivator = row['경작자'] if pd.notna(row['경작자']) else ""
        return f"{row['코드번호']}<br>({cultivator})"

df_jc['X_Label'] = df_jc.apply(get_xlabel, axis=1)

# (4) 정렬: 일반 필지 먼저, 농협(비교군)을 맨 뒤로 보내기 위해 정렬
df_jc['sort_key'] = df_jc['코드번호'] == '진천 농협' # False(0) -> True(1) 순
df_jc = df_jc.sort_values(by=['sort_key', '코드번호'])

# ---------------------------------------------------------
# 3. Plotly 그래프 시각화
# ---------------------------------------------------------

# 색상 매핑 (진천 농협: 빨강, 나머지는 품종별 색상)
color_map = {
    '진천 농협 (비교군)': 'red',      # 요청하신 비교군 강조색
    '황금노들': '#1f77b4',           # 파란색 계열
    '청풍': '#2ca02c'                # 초록색 계열
}

fig = px.bar(
    df_jc,
    x='X_Label',               # X축: 필지코드(경작자)
    y='건조 수확량(kg/10a)',    # Y축: 수확량
    color='구분',              # 색상: 품종 및 농협 구분
    text_auto='.1f',           # 값 표시 (소수점 1자리)
    title='진천 지역 필지별 수확량 비교 (품종/경작자 구분)',
    color_discrete_map=color_map, # 커스텀 색상 적용
    hover_data=['품종', '경작자', '건조 수확량(kg/10a)'] # 마우스 오버 시 상세 정보
)

# 레이아웃 다듬기
fig.update_layout(
    xaxis_title="필지코드 (경작자)",
    yaxis_title="건조 수확량 (kg/10a)",
    legend_title_text='구분 (품종)',
    xaxis={'categoryorder': 'array', 'categoryarray': df_jc['X_Label'].tolist()}, # 정렬 순서 고정
    template='plotly_white' # 깔끔한 흰색 배경
)

fig.show()

In [4]:
image_filename = "진천 필지별 수확량.png"
fig.write_image(
              image_filename,
              width=1200,
              height=600,
              scale=2  # 2배 해상도
            )

In [9]:
import plotly.express as px
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. 데이터 준비 (이전 단계의 df_jc 데이터가 있다고 가정)
# ---------------------------------------------------------
# 만약 이전 코드를 실행하지 않았다면 아래 주석을 풀어 df_jc를 다시 생성하세요.
# (df_jc 생성 로직은 동일합니다)
# ... (생략: 위 코드와 동일한 df_jc 생성 및 전처리 과정) ...

# 데이터가 끊기지 않게 안전하게 다시 한 번 핵심 전처리를 수행합니다.
# (이미 df_jc가 있다면 이 부분은 건너뛰어도 됩니다)
if 'df_jc' not in locals():
    # 앞선 코드의 로직 복원
    df_jc = df[df['지역'] == '진천'].copy()
    mask_nonghyup = (df_jc['경작자'] == '농협 평균') | (df_jc['코드번호'].isna())
    df_jc.loc[mask_nonghyup, '코드번호'] = '진천 농협'

    def get_xlabel(row):
        if row['코드번호'] == '진천 농협': return '진천 농협'
        cultivator = row['경작자'] if pd.notna(row['경작자']) else ""
        return f"{row['코드번호']}<br>({cultivator})"
    df_jc['X_Label'] = df_jc.apply(get_xlabel, axis=1)

# ---------------------------------------------------------
# 2. 차이값(Difference) 계산 로직
# ---------------------------------------------------------

# (1) 기준값(Baseline) 추출: 진천 농협의 수확량
baseline_row = df_jc[df_jc['코드번호'] == '진천 농협']
if not baseline_row.empty:
    baseline_value = baseline_row['건조 수확량(kg/10a)'].values[0]
else:
    baseline_value = 0 # 예외 처리

# (2) 비교 대상 데이터 추출 (진천 농협 제외)
df_diff = df_jc[df_jc['코드번호'] != '진천 농협'].copy()

# (3) 차이 계산 (필지 수확량 - 농협 평균)
df_diff['수확량 차이(kg)'] = df_diff['건조 수확량(kg/10a)'] - baseline_value

# (4) 색상 구분을 위한 컬럼 생성 (양수: 초과, 음수: 미달)
df_diff['상태'] = np.where(df_diff['수확량 차이(kg)'] >= 0, '평균 이상 (+)', '평균 미만 (-)')
df_diff['label_text'] = df_diff['수확량 차이(kg)'].apply(lambda x: f'{x:+.1f}')

# ---------------------------------------------------------
# 3. 시각화 (Diverging Bar Chart)
# ---------------------------------------------------------

# 색상 매핑 (양수: 파랑/초록, 음수: 빨강/주황)
color_map_diff = {
    '평균 이상 (+)': '#1f77b4',  # 파란색 (긍정)
    '평균 미만 (-)': '#d62728'   # 빨간색 (부정)
}

fig = px.bar(
    df_diff,
    x='X_Label',
    y='수확량 차이(kg)',
    color='상태',
    color_discrete_map=color_map_diff,
    # text_auto='+.1f', # 부호(+/-)와 소수점 1자리 표시
    text ='label_text',
    title=f'진천 농협 평균({baseline_value:.1f}kg) 대비 필지별 수확량 차이',
)

# ---------------------------------------------------------
# 4. 레이아웃 및 기준선 설정
# ---------------------------------------------------------
fig.update_layout(
    xaxis_title="필지 (경작자)",
    yaxis_title="수확량 차이 (kg/10a)",
    legend_title_text="상태",
    template='plotly_white',

    # ★ 핵심: X축 순서를 데이터프레임의 정렬 순서대로 고정 ★
    xaxis={
        'categoryorder': 'array',
        'categoryarray': df_diff['X_Label'].tolist()
    },

    # 0점 기준선
    shapes=[dict(type="line", xref="paper", yref="y",
                 x0=0, y0=0, x1=1, y1=0,
                 line=dict(color="black", width=2, dash="dash"))]
)

fig.update_traces(textposition='outside')

fig.show()

In [10]:
image_filename = "진천 평균대비 수확량 차이.png"
fig.write_image(
              image_filename,
              width=1200,
              height=600,
              scale=2  # 2배 해상도
            )